## Limpieza de datos de un E-commerce

### Contexto:

Una tienda online desea analizar las ventas del último año, pero los datos provienen de múltiples fuentes y contienen errores comunes como valores nulos, duplicados y formatos inconsistentes.

---

### Consigna:

Utiliza **Pandas** para limpiar el archivo `ventas_ecomerce.csv`, aplicando técnicas de detección de duplicados, reemplazo de valores erróneos y normalización de formatos.

---

### Paso a paso:

1. Carga el dataset con `pd.read_csv()`.
2. Detecta y elimina filas duplicadas.
3. Identifica columnas con valores nulos y decide si deben eliminarse o imputarse.
4. Reemplaza valores incorrectos en la columna **Método de pago** (ej: efectivo, tarjeta) usando `replace()`.
5. Asegúrate de que las fechas estén en formato `datetime` y los precios como `float`.

---

### Archivo de trabajo:
Sube y utiliza el archivo `ventas_ecomerce.csv`.


In [1]:
# Conectar Google Colab con Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Paso 1: Importar librerías necesarias
import pandas as pd

# Conectar Google Colab con Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Paso 2: Cargar el archivo CSV
df = pd.read_csv('/content/drive/MyDrive/CURSO FUNDAMENTOS DE LA INGENIERIA DE DATOS/ventas_ecomerce.csv')

# Paso 3: Eliminar filas duplicadas
df = df.drop_duplicates()

# Paso 4: Identificar y manejar valores nulos
print("Valores nulos por columna antes de limpieza:")
print(df.isnull().sum())

# Eliminamos filas con valores nulos
df = df.dropna()

print("Valores nulos por columna después de limpieza:")
print(df.isnull().sum())

# Paso 5: Reemplazar valores incorrectos en 'metodo_pago'
df['metodo_pago'] = df['metodo_pago'].replace({
    'tarjeta': 'Tarjeta',
    'Tarjeta de crédito': 'Tarjeta',
    'efectivo': 'Efectivo',
    'Efect': 'Efectivo',
    'transferencia': 'Transferencia',
    'transf': 'Transferencia'
})

# Paso 6: Asegurar formatos correctos

# Convertir columna 'fecha' a tipo datetime
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

# Convertir columna 'precio_unitario' a tipo float
df['precio_unitario'] = pd.to_numeric(df['precio_unitario'], errors='coerce')

# Convertir columna 'cantidad' a tipo int
df['cantidad'] = pd.to_numeric(df['cantidad'], errors='coerce')

# Paso 7: Calcular total de venta (opcional)
df['total_venta'] = df['precio_unitario'] * df['cantidad']



# Rellenar fechas NaT con forward fill
df['fecha'] = df['fecha'].ffill()

# Si quedan fechas NaT al inicio, rellenar con backward fill
df['fecha'] = df['fecha'].bfill()


# Ver primeras filas del dataframe limpio
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Valores nulos por columna antes de limpieza:
id_venta           0
fecha              0
cliente            1
producto           0
cantidad           0
precio_unitario    0
total              0
metodo_pago        5
direccion_envio    0
email              0
estado_envio       0
dtype: int64
Valores nulos por columna después de limpieza:
id_venta           0
fecha              0
cliente            0
producto           0
cantidad           0
precio_unitario    0
total              0
metodo_pago        0
direccion_envio    0
email              0
estado_envio       0
dtype: int64


,id_venta,fecha,cliente,producto,cantidad,precio_unitario,total,metodo_pago,direccion_envio,email,estado_envio,total_venta
0,1,2024-07-01,Carlos Ruiz,Silla Gamer,4,475.85,1903.40,Efectivoo,Valparaíso,cliente0@ejemplo.com,Cancelado,1903.40
1,2,2024-07-01,María Díaz,Silla Gamer,3,235.03,705.09,Transferencia,Rancagua,cliente1@ejemplo.com,Pendiente,705.09
2,3,2024-07-01,María Díaz,Webcam,2,114.05,228.10,Efectivoo,Santiago,cliente2@ejemplo.com,enviado,228.10
3,4,2024-07-01,Juan Pérez,Mouse,2,13.46,26.92,Transferencia,Concepción,cliente3@ejemplo.com,Pendiente,26.92
4,5,2024-07-01,Luis Gómez,Webcam,4,189.52,758.08,Tarjeta,La Serena,cliente4@ejemplo.com,Enviado,758.08


## Transformación y enriquecimiento de datos de Recursos Humanos

### Contexto:
 El área de Recursos Humanos necesita preparar un informe de rotación laboral con datos de empleados.
 Se requiere agrupar, discretizar y enriquecer la información.

#### Consigna:
 Aplica transformaciones avanzadas a un dataset de empleados (archivo 'empleados.csv')
 para generar nuevas variables y preparar los datos para el análisis.

### Paso a paso:
 1. Agrupa los empleados por área y calcula el promedio de antigüedad y edad.
 2. Discretiza la edad en categorías:
    - Joven (<= 30)
    - Medio (31 - 45)
    - Senior (46+)
 3. Crea una nueva columna usando apply() para clasificar a los empleados según su permanencia:
    - Antigüedad > 5 años → "estable"
    - Antigüedad <= 5 años → "nuevo"
 4. Exporta el dataframe limpio con to_csv().


In [12]:
import pandas as pd

# Paso 1: Cargar el archivo empleados.csv
df = pd.read_csv('/content/drive/MyDrive/CURSO FUNDAMENTOS DE LA INGENIERIA DE DATOS/empleados.csv')

# Paso 2: Agrupar por área y calcular promedio de antigüedad y edad
promedios = df.groupby('area')[['antiguedad', 'edad']].mean().reset_index()
print("Promedio de antigüedad y edad por área:")
print(promedios)

# Paso 3: Crear función para categorizar edad
def edad_categoria(edad):
    if edad <= 30:
        return 'Joven'
    elif edad <= 45:
        return 'Medio'
    else:
        return 'Senior'

# Aplicar función a la columna edad
df['categoria_edad'] = df['edad'].apply(edad_categoria)

# Paso 4: Crear función para clasificar permanencia
def clasificar_antiguedad(antiguedad):
    if antiguedad > 5:
        return 'estable'
    else:
        return 'nuevo'

# Aplicar función a la columna antiguedad
df['clasificacion'] = df['antiguedad'].apply(clasificar_antiguedad)

# Mostrar los primeros datos con nuevas columnas
print("\nDatos con categorías y clasificación:")
print(df.head())

# Paso 5: Guardar el dataframe limpio a un nuevo archivo CSV
df.to_csv('/content/empleados_transformados.csv', index=False)
print("\nArchivo guardado como 'empleados_transformados.csv'")


Promedio de antigüedad y edad por área:
        area  antiguedad       edad
0   Finanzas   12.000000  45.833333
1         IT   18.400000  42.200000
2  Marketing    9.142857  33.571429
3       RRHH    8.571429  35.000000
4     Ventas   13.400000  47.600000

Datos con categorías y clasificación:
   empleado_id nombre       area  edad  antiguedad fecha_ingreso  estado  \
0            1    Ana     Ventas    60          28    1997-08-01  Activo   
1            2   Luis       RRHH    36          10    2015-07-28  Activo   
2            3  Carla  Marketing    29           4    2021-07-26  Activo   
3            4  Jorge     Ventas    60          18    2007-07-30  Activo   
4            5  Marta  Marketing    44          10    2015-07-28  Activo   

                puesto  salario categoria_edad clasificacion  
0       Jefe de ventas     1750         Senior       estable  
1     Analista de RRHH     2092          Medio       estable  
2    Community Manager     1432          Joven         nuev